In [ ]:
path_root = '/content/'
path_ipynb = path_root + 'ipynb/'
path_py = path_root + 'py/'
path_test = path_root + 'test/'
path_gen = path_root + 'gen/'
path_moss = path_root + 'moss'
path_moss_base = path_py + '...'
path_src = 'source.c'
ext_in = '.in'
ext_out = '.out'
workbook_url = '...'
sheet_read = 'Radovi'
sheet_write = 'Acinonyx'
sheet_write_header = ['Ime i prezime', 'Broj indeksa']
moss_user_id = 284199714
index_name = 2
index_number = 3
index_ipynb = 5
total_submissions = 1000
cmd_compile_gen = 'make {{file_gen}}'
cmd_run_gen = './{{file_gen}}'
cmd_run_exe = 'python {{file_exe}} {{file_src}} {{file_gen}}'
cmd_timeout = 3

In [ ]:
!rm -rf '{path_ipynb}' '{path_py}' '{path_gen}'
!mkdir '{path_ipynb}' '{path_py}' '{path_gen}'

In [ ]:
!pip install gspread
!pip install unidecode

In [ ]:
import os
import re
import math
import json
import gspread
import unidecode
from subprocess import *
from io import TextIOWrapper
from contextlib import suppress
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

book = gspread.authorize(gauth.credentials).open_by_url(workbook_url)
with suppress(gspread.WorksheetNotFound): book.del_worksheet(book.worksheet(sheet_write))
results = book.add_worksheet(sheet_write, total_submissions, len(os.listdir(path_test)) * 2 + 2)

header = []
header.extend(sheet_write_header)
header_test_dir = ['test/' + test for test in sorted(os.listdir(path_test))]
header_test = [f'{h} (EXE)' for h in header_test_dir]
header_test.extend([f'{h} (GEN)' for h in header_test_dir])
header.extend(sorted(header_test))
for i, col in enumerate(header):
    results.update_cell(1, i + 1, col)

for i, row in enumerate(book.worksheet(sheet_read).get_all_values()[1:]):
    # Downloading
    name, number, ipynb = row[index_name], row[index_number], row[index_ipynb]

    index = str(i + 1).zfill(int(math.log10(total_submissions)))
    title_unicode = re.sub('[\s/]', '_', f'{index}_{name}_{number}')
    title = unidecode.unidecode(title_unicode)

    print(f"[{i+1}]\tDownloading\t{title}.ipynb <- {ipynb}")

    id_ipynb = re.findall('[-\w]{25,}', ipynb)[0]
    file_ipynb = f'{path_ipynb}{title}.ipynb'
    file_py = f'{path_py}{title}.py'

    downloaded = drive.CreateFile({'id':id_ipynb})
    downloaded.GetContentFile(file_ipynb)

    # Converting
    with open(file_ipynb, 'r') as fd_json:
        json_ipynb = json.load(fd_json)

        print(f"[{i+1}]\tConverting\t{title}.py <- {title}.ipynb")

        with open(file_py, 'w') as fd_py:
            code = [line for cell in json_ipynb['cells'] if cell['cell_type'] == 'code' for line in cell['source']]
            fd_py.write(''.join(code))
    
    # Testing
    print(f"[{i+1}]\tTesting\t\t{title}.py")

    results.update_cell(i + 2, 1, name)
    results.update_cell(i + 2, 2, number)

    for j, file_test in enumerate(sorted(os.listdir(path_test))):
        file_test = os.path.join(path_test, file_test)
    
        file_exe, file_src, file_gen = file_py, f'{file_test}/{path_src}', f'{path_gen}{title}.py'

        run_exe = re.sub('{{file_exe}}', file_exe, cmd_run_exe)
        run_exe = re.sub('{{file_src}}', file_src, run_exe)
        run_exe = re.sub('{{file_gen}}', file_gen, run_exe)

        compile_gen = re.sub('{{file_gen}}', file_gen, cmd_compile_gen)
        run_gen = re.sub('{{file_gen}}', file_gen, cmd_run_gen)

        points_exe, points_gen = 0, 0

        for file_input in sorted(os.listdir(file_test)):
            if not file_input.endswith(ext_in):
                continue

            file_input = os.path.join(file_test, file_input)
            file_output = re.sub(f'(?<=\d){ext_in}', ext_out, file_input)

            with open(file_input, 'r') as fd_input, open(file_output, 'r') as fd_output:
                test_input, test_output = fd_input.read(), fd_output.read()

                def run_test(cmd, input):
                    return run(cmd.split(),
                               input=input.encode(),
                               stdout=PIPE,
                               check=True,
                               timeout=cmd_timeout).stdout.decode('utf-8')

                output = run_test(run_exe, test_input)
                points_exe += 1 if output == test_output else 0

                run(compile_gen.split())
                output = run_test(run_gen, test_input)
                points_gen += 1 if output == test_output else 0

        results.update_cell(i + 2, j + 3, points_exe)
        results.update_cell(i + 2, j + 4, points_gen)

print(f"Results\t{workbook_url}")

In [ ]:
MOSS = r"""#!/usr/bin/env perl
use IO::Socket;
@languages = ("c", "cc", "java", "ml", "pascal", "ada", "lisp",
"scheme", "haskell", "fortran", "ascii", "vhdl", "perl", "matlab",
"python", "mips", "prolog", "spice", "vb", "csharp", "modula2",
"a8086", "javascript", "plsql", "verilog");
$server = 'moss.stanford.edu';
$port = '7690';
$noreq = "Request not sent.";
$usage = "Usage: moss [-x] [-l language] [-d] [-b basefile1] ...
[-b basefilen] [-m #] [-c \"string\"] file1 file2 file3 ...";
$userid = {{moss_user_id}};
$opt_l = "c";
$opt_m = 10;
$opt_d = 0;
$opt_x = 0;
$opt_c = "";
$opt_n = 250;
$bindex = 0;
while(@ARGV && ($_ = $ARGV[0]) =~ /^-(.)(.*)/) {
    ($first, $rest) = ($1, $2);
    shift(@ARGV);
    if($first eq "d") {
        $opt_d = 1;
        next;
    }
    if($first eq "b") {
        if($rest eq '') {
            die "No argument for option -b.\n" unless @ARGV;
            $rest = shift(@ARGV);
        }
        $opt_b[$bindex++] = $rest;
        next;
    }
    if($first eq "l") {
        if($rest eq '') {
	        die "No argument for option -l.\n" unless @ARGV;
	        $rest = shift(@ARGV);
	    }
	    $opt_l = $rest;
        next;
    }
    if($first eq "m") {
        if($rest eq '') {
            die "No argument for option -m.\n" unless @ARGV;
            $rest = shift(@ARGV);
	    }
        $opt_m = $rest;
        next;
    }
    if($first eq "c") {
	    if($rest eq '') {
	        die "No argument for option -c.\n" unless @ARGV;
	        $rest = shift(@ARGV);
	    }
	    $opt_c = $rest;
	    next;
    }
    if($first eq "n") {
	    if($rest eq '') {
	        die "No argument for option -n.\n" unless @ARGV;
	        $rest = shift(@ARGV);
	    }
	    $opt_n = $rest;
	    next;
    }
    if($first eq "x") {
	    $opt_x = 1;
	    next;
    }
    if($first eq "s") {
	    $server = shift(@ARGV);
	    next;
    }
    if($first eq "p") {
	    $port = shift(@ARGV);
	    next;
    }
    die "Unrecognized option -$first. $usage\n"; 
}
print "Checking files . . . \n";
$i = 0;
while($i < $bindex) {
    die "Base file $opt_b[$i] does not exist. $noreq\n" unless -e "$opt_b[$i]";
    die "Base file $opt_b[$i] is not readable. $noreq\n" unless -r "$opt_b[$i]";
    die "Base file $opt_b is not a text file. $noreq\n" unless -T "$opt_b[$i]";
    $i++;
}
foreach $file (@ARGV) {
    die "File $file does not exist. $noreq\n" unless -e "$file";
    die "File $file is not readable. $noreq\n" unless -r "$file";
    die "File $file is not a text file. $noreq\n" unless -T "$file";
}
if("@ARGV" eq '') {
   die "No files submitted.\n $usage";
}
print "OK\n";
$sock = new IO::Socket::INET(PeerAddr => $server, PeerPort => $port, Proto => 'tcp');
die "Could not connect to server $server: $!\n" unless $sock;
$sock->autoflush(1);
sub read_from_server {
    $msg = <$sock>;
    print $msg;
}
sub upload_file {
    local ($file, $id, $lang) = @_;
    open(F, $file);
    $size = 0;
    while(<F>) {
	    $size += length($_);
    }
    close(F);
    print "Uploading $file ...";
    open(F, $file);
    $file =~s/\s/\_/g;
    print $sock "file $id $lang $size $file\n";
    while(<F>) {
        print $sock $_;
    }
    close(F);
    print "done.\n";
}
print $sock "moss $userid\n";
print $sock "directory $opt_d\n";
print $sock "X $opt_x\n";
print $sock "maxmatches $opt_m\n";
print $sock "show $opt_n\n";
print $sock "language $opt_l\n";
$msg = <$sock>;
chop($msg);
if($msg eq "no") {
    print $sock "end\n";
    die "Unrecognized language $opt_l.";
}
$i = 0;
while($i < $bindex) {
    &upload_file($opt_b[$i++], 0, $opt_l);
}
$setid = 1;
foreach $file (@ARGV) {
    &upload_file($file, $setid++, $opt_l); 
}
print $sock "query 0 $opt_c\n";
print "Query submitted. Waiting for the server's response.\n";
&read_from_server();
print $sock "end\n";
close($sock);
"""

with open(path_moss, 'w') as fd_moss:
    moss = re.sub('{{moss_user_id}}', str(moss_user_id), MOSS)
    fd_moss.write(moss)

!chmod +x moss
!./moss -l python -b '{path_moss_base}' {path_py}*.py